In [106]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import seaborn as sns

In [294]:
def load_census_data(directory):
    # Load census data from given directory
    df = []
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith('.csv'):
            df_temp = pd.read_csv(os.path.join(directory,filename),skiprows=1)
            df_temp["Year_ID"] = filename[:4] + df_temp['Id2'].map(str)
            df_temp = df_temp.set_index("Year_ID")
            df.append(df_temp)


    census_data = pd.concat(df)
    return census_data

In [295]:
census_data_2013 = load_census_data("data/census_data_2013/")

/Users/admin/miniconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  del sys.path[0]


In [296]:
estimates = [c for c in census_data_2013.columns if c[:9] == "Estimate;"]
census_estimates_2013 = census_data_2013[estimates]
census_estimates_2013 = census_estimates_2013.replace("(X)",np.nan)
census_estimates_2013 = census_estimates_2013.fillna(0)

census_estimates_2013

,Estimate; ANCESTRY - Total population,Estimate; ANCESTRY - Total population - American,Estimate; ANCESTRY - Total population - Arab,Estimate; ANCESTRY - Total population - Czech,Estimate; ANCESTRY - Total population - Danish,Estimate; ANCESTRY - Total population - Dutch,Estimate; ANCESTRY - Total population - English,Estimate; ANCESTRY - Total population - French (except Basque),Estimate; ANCESTRY - Total population - French Canadian,Estimate; ANCESTRY - Total population - German,...,"Estimate; WORLD REGION OF BIRTH OF FOREIGN BORN - Foreign-born population, excluding population born at sea - Latin America","Estimate; WORLD REGION OF BIRTH OF FOREIGN BORN - Foreign-born population, excluding population born at sea - Northern America","Estimate; WORLD REGION OF BIRTH OF FOREIGN BORN - Foreign-born population, excluding population born at sea - Oceania",Estimate; YEAR OF ENTRY - Foreign born,Estimate; YEAR OF ENTRY - Foreign born - Entered 2010 or later,Estimate; YEAR OF ENTRY - Foreign born - Entered before 2010,Estimate; YEAR OF ENTRY - Native,Estimate; YEAR OF ENTRY - Native - Entered 2010 or later,Estimate; YEAR OF ENTRY - Native - Entered before 2010,Estimate; YEAR OF ENTRY - Population born outside the United States
Year_ID,,,,,,,,,,,,,,,,,,,,,
201521001,18852,3413,17,0,26,144,4386,362,36,1612,...,94,31,0,243,45,198,61,8,53,304
201521003,20355,3477,10,25,0,424,1956,336,11,2599,...,123,14,12,238,17,221,289,0,289,527
201521005,21761,4576,0,23,101,158,2801,368,44,2818,...,125,0,0,198,11,187,118,64,54,316
201521007,8256,1629,10,25,0,117,679,243,1,910,...,24,0,0,152,10,142,53,0,53,205
201521009,42925,9496,20,48,61,550,5639,716,108,3677,...,417,89,0,752,164,588,114,11,103,866
201521011,11978,2967,0,0,0,90,1122,79,2,1054,...,87,0,0,90,0,90,29,0,29,119
201521013,27950,7465,4,0,5,94,1347,168,6,1263,...,48,0,0,122,48,74,100,0,100,222
201521015,124617,11675,436,185,218,1301,12904,2309,410,38474,...,1869,334,73,6293,997,5296,988,162,826,7281
201521017,20013,5919,31,15,41,300,1949,281,79,2349,...,777,15,5,981,91,890,35,0,35,1016


In [314]:
def load_drug_data():
    drug_data = pd.read_excel('data/drug_data.xlsx', sheet_name = "Data")

    drug_data["Year_ID"] = drug_data["YYYY"].map(str) + drug_data["FIPS_Combined"].map(str)
    drug_data = drug_data.set_index("Year_ID")


    drug_piv = drug_data.pivot(columns="SubstanceName", values=["DrugReports"])

    drug_piv[["TotalDrugReportsCounty","TotalDrugReportsState"]] = drug_data.drop_duplicates(subset=["TotalDrugReportsCounty","TotalDrugReportsState"])[["TotalDrugReportsCounty","TotalDrugReportsState"]]

    drug_piv["Total"] = drug_piv["DrugReports"].sum(axis=1)
    
    return drug_piv

In [315]:
drug_data = load_drug_data()

In [316]:
drug_data_2013 = drug_data.loc[census_estimates_2013.index]

/Users/admin/miniconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


In [320]:
drug_data_2013["Total"]

Year_ID
201521001     16.0
201521003     12.0
201521005     17.0
201521007      1.0
201521009     47.0
201521011     74.0
201521013    329.0
201521015    361.0
201521017     16.0
201521019    203.0
201521021     53.0
201521023     16.0
201521025     30.0
201521027      1.0
201521029     88.0
201521031     12.0
201521033     16.0
201521035     18.0
201521037    634.0
201521039      1.0
201521041     56.0
201521043     83.0
201521045     33.0
201521047     37.0
201521049    109.0
201521051     66.0
201521053     22.0
201521055      5.0
201521057     10.0
201521059     56.0
             ...  
201354051     93.0
201354053      6.0
201354055    322.0
201354057      3.0
201354059     94.0
201354061     52.0
201354063      7.0
201354065     13.0
201354067    228.0
201354069    173.0
201354071      NaN
201354073      3.0
201354075     19.0
201354077     12.0
201354079     45.0
201354081    358.0
201354083     34.0
201354085      1.0
201354087      NaN
201354089     89.0
201354091      2.0
2013